# Pandas Project - Alberto Ibarra

### Importar librerias para la limpieza y Cargar base de datos

In [15]:
import pandas as pd
import numpy as np
import re

In [16]:
df_raw_data = pd.read_csv('attacks.csv', encoding='latin-1')

In [17]:
print('Registros iniciales:',df_raw_data.shape[0])

Registros iniciales: 25723


In [18]:
df_raw_data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [19]:
df_raw_data = df_raw_data.rename(columns= {'Case Number':'Case','Sex ':'Sex','Species ':'Species','Fatal (Y/N)':'Fatal','Investigator or Source':'Source'})

# Limpieza inicial de filas

### Quitar filas duplicadas

In [20]:
df_lessdup_data=df_raw_data.copy()
df_lessdup_data=df_lessdup_data.drop_duplicates()
print('Filas descartadas por duplicacion:',df_raw_data.shape[0]-df_lessdup_data.shape[0])
print('Registros actuales:',df_lessdup_data.shape[0])

Filas descartadas por duplicacion: 19411
Registros actuales: 6312


### Detección de cantidad de datos invalidos

In [21]:
Datosvalidos = df_lessdup_data.isnull().sum()
Datosvalidos

Case                 2
Date                10
Year                12
Type                14
Country             60
Area               465
Location           550
Activity           554
Name               220
Sex                575
Age               2841
Injury              38
Fatal              549
Time              3364
Species           2848
Source              27
pdf                 10
href formula        11
href                10
Case Number.1       10
Case Number.2       10
original order       3
Unnamed: 22       6311
Unnamed: 23       6310
dtype: int64

### Se detrmina como datos muy relevantes: caso, fecha, año, tipo, pais y daño, por lo que las filas que no cuenten con alguno de estos datos serán descartadas

In [22]:
df_row_cleaned=df_lessdup_data.dropna(subset=['Case','Date','Year','Type','Country','Injury'])

In [23]:
print('Filas descartadas:', df_lessdup_data.shape[0] - df_row_cleaned.shape[0])
print('Registros Actuales:', df_row_cleaned.shape[0])

Filas descartadas: 93
Registros Actuales: 6219


In [29]:
Datosvalidos = df_row_cleaned.isnull().sum()
#Datosvalidos

## Asignacion de valores a todos los datos no validos

In [30]:
c_fill_ukn = {'Area':'UNKNOWN','Location':'UNKNOWN','Activity':'UNKNOWN','Name':'UNKNOWN','Sex': '?','Age':0,'Fatal':'?','Time':'?','Species':'UNKNOWN','Source':'UNKNOWN','href formula':'UNKNOWN','Unnamed: 22':0,'Unnamed: 23':0}
df_row_cleaned.fillna(c_fill_ukn,inplace=True,downcast=True)
#df_row_cleaned.isnull().sum()

# Limpieza de columnas

## Limpieza columna Case

In [93]:
#Se les dará un numero de caso único segun su aparicion en la lista comenzando del final al principio comenzando con 
#las letras 'SA' - Shark Atack y rellenando con ceros las 5 primeras posiciones

df_col01_clean = df_row_cleaned.copy()
df_col01_clean.reset_index(drop=True, inplace=True)

for x in range(len(df_col01_clean['Case'])):
    nceros= 5 - len(str(len(df_col01_clean['Case']) - x))
    ceros = ''
    for i in range(nceros):
        ceros += '0'
    df_col01_clean.iloc[x,0] = 'SA'+ ceros +str(len(df_col01_clean['Case']) - x)

In [94]:
display(df_col01_clean.head(2))
display(df_col01_clean.tail(2))

,Case,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,SA06219,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,0,0
1,SA06218,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,UNKNOWN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,0,0


,Case,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
6217,SA00002,1883-1889,0.0,Unprovoked,PANAMA,UNKNOWN,"Panama Bay 8ºN, 79ºW",UNKNOWN,Jules Patterson,M,...,UNKNOWN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,0,0
6218,SA00001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,...,UNKNOWN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2.0,0,0


## Limpieza columna Date

In [98]:
df_col02_clean = df_col01_clean.copy()
pd.options.display.max_rows = 4000

hacer lista de meses para comparacion
buscar con la lista de meses y extraerlos en una columna nueva
quitar todas las palabras que aparezcan antes del primero numero
extraer los dos  o un primeros numeros y asignarlo como dia en una nueva columna
quitar dias y meses 

-Revisar
Buscar años que quedan y borrarlos hasta 1900
Buscar registros sin año y asignar el que viene en la fecha
Asignar "UNKNOWN" a los registros que no tengan indicio de la fecha




In [104]:
df_col02_clean.head()

,Case,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,SA06219,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,0,0
1,SA06218,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,UNKNOWN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,0,0
2,SA06217,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,UNKNOWN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,0,0
3,SA06216,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,0,0
4,SA06215,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,0,0


## Limpieza columna Year

## Limpieza columna Type

## Limpieza columna Country

In [ ]:
df_col1_clean.head()
series_country = df_col1_clean['Country'].copy()

In [ ]:
#Limpieza de la base de datos de paises
df_world = pd.read_csv('countries.csv')
series_countries_valid = df_world[['Country']].copy()  #Copia de la serie
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.upper()) #Mayusculas
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.replace(' ','')) #Espacios
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.replace(',','')) #Comas
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.replace('BAHAMASTHE','BAHAMAS')) #Comas
#series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: re.sub(r"^\s+|\s+$",'',x)) #Quita espacios al principio y al final
list_countries_valid = list(series_countries_valid['Country'])

In [ ]:
#list_countries_valid

In [ ]:
#Funcion para buscar en la cadena dada un de los posibles paises, si hay dos paises tomará el primero en caso de
#no encontrarlo regresara el mismo dato con una "X-" para decidir el siguiente criterio de limpieza
def iscountry(x):
    for i in range(len(list_countries_valid)):
        if x.find(list_countries_valid[i])>=0:
            return list_countries_valid[i]
        else:
            pass
    return 'X-' + x

In [ ]:
#Limpieza de Series ·Countries·
series_country = series_country.apply(lambda x: str(x)) #Transformacion a str para evitar error de tipo de datos
series_country = series_country.apply(lambda x: x.upper()) #Hologacion a mayusculas
series_country = series_country.apply(lambda x: x.replace(' ','')) #Quitar espacios
series_country = series_country.apply(lambda x: x.replace('USA','UNITEDSTATES')) #Excepcion pais con dos nombres
series_country = series_country.apply(lambda x: x.replace('ENGLAND','UNITEDKINGDOM')) #Excepcion pais con dos nombres
series_country = series_country.apply(lambda x: iscountry(x))#Aplicacion de la funcion iscountry que homologa el nomnre de los paises

In [ ]:
#Asignacion de la limpieza a la tabla general
df_col2_clean=df_col1_clean.copy()
df_col2_clean['Country']=series_country

In [ ]:
b1=series_country.value_counts()
lst_b1=[]
reg_count=0
for x in range(len(b1)):
    if b1.index[x].find('X-')==0:
        lst_b1.append((b1.index[x],b1[x]))
        reg_count+= b1[x] 
#print(lst_b1)
print('Casos con pais mal clasificado:',reg_count)

## Limpieza columna Area

## Limpieza columna Location

## Limpieza columna Activity

## Limpieza columna Sex

## Limpieza columna Age

## Limpieza columna Injury

## Limpieza columna Fatal

## Limpieza columna Time

## Limpieza columna Species

In [ ]:
#Dimensionando el tamaño del problema
series_nasty_species=df_row_cleaned['Species'].unique()
print(series_nasty_species.shape[0], 'clasificaciones diferntes')

In [ ]:
#Carga y limpieza de base de datos con nombre de especies de tiburones para comparar
df_shark_types = pd.read_csv('shark_types.csv')
df_shark_types=df_shark_types.drop_duplicates()
series_shark_types=df_shark_types['English']
series_shark_types=series_shark_types.dropna()
series_shark_types=series_shark_types.apply(lambda x: str(x))
series_shark_types=series_shark_types.apply(lambda x: x.upper())
series_shark_types=series_shark_types.apply(lambda x: x.replace('SHARK',''))
series_shark_types=series_shark_types.apply(lambda x: x.replace("'",''))
series_shark_types=series_shark_types.apply(lambda x: re.sub(r'\s', '', x))
series_shark_types=series_shark_types.drop_duplicates()
list_shark_species = list(series_shark_types)

In [ ]:
#Funcion para buscar en la cadena dada un de los posibles tipos de tiburones
def isshark(x):
    for i in range(len(list_shark_species)):
        if x.find(list_shark_species[i])>=0:
            return list_shark_species[i]
        else:
            pass
    return 'UNKNOWN'

In [ ]:
#Limpieza de Series ·Species·
series_species = df_row_cleaned['Species']
series_species = series_species.fillna('UNKNOWN') #Relleno de NA por la palabra UNKNOWN para eliminar Nulos
series_species = series_species.apply(lambda x: str(x)) #Transformacion a str para evitar error de tipo de datos
series_species = series_species.apply(lambda x: x.replace(' ',''))#Eliminacion de espacios
series_species = series_species.apply(lambda x: x.upper()) #Hologacion a mayusculas
series_species = series_species.apply(lambda x: x.replace('SHARK','')) #Eliminacion de la palabra "Shark"
series_species = series_species.apply(lambda x: x.replace('"','')) #Eliminacion de comillas dobles
series_species = series_species.apply(lambda x: isshark(x)) #Aplicacion de la funcion isshark

In [ ]:
nasty_col_species = series_species.unique()
print('Especies identificadas: ', nasty_col_species.shape[0]-1)
#series_species.value_counts()

In [ ]:
#Integracion de la Series ·Species· al data frame
df_col1_clean = df_row_cleaned.copy()
df_col1_clean['Species'] = series_species